Задание из 2-х частей.
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95
2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

import warnings 
warnings.filterwarnings("ignore", category=Warning)

## 0. Загрузка и обзор данных

In [2]:
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
data['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [4]:
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)

In [5]:
data['target'].value_counts()

1    16724
0     3024
Name: target, dtype: int64

**Положительные отзывы:**

In [6]:
data.loc[data['target'] == 1, 'Content'].sample(20, random_state=42)

19216                                  Спасибо, все удобно
12380                                                   Ok
953                                                Отлично
15329                                     Удобное спасибо!
12421                                  Отличное приложение
17805                                                   Ок
8245                                                  норм
7299                                               отлично
5504                              Очень удобное приложение
19928                                              Отлично
12680    Заходить в приложение стало в 2 раза дольше,фу...
7209                Не могу скачать приложение, ошибка 963
12721                 Всё отлично!приложение очень радует!
10057                                Удобно.Быстро.Надежно
3191                                Очень просто и быстро.
5886     Мне понравилось очень удобно сидиш дома и все ...
10309    Перестало заходить, причем именно дома по wi f.

**Отрицательные отзывы:**

In [7]:
data.loc[data['target'] == 0, 'Content'].sample(20, random_state=42)

20636                     Дайте доступ к роот правам, с*ки
2226     По три раза просит пароль задолбал исправьте п...
13980    Ну задолбало уже по 2-3 раза вводить пароль дл...
13173    Если честно - дерьмо Ваше приложение! Исправьт...
8452                             Не приходит СМС с паролем
10162    В шаблонах чехарда появляются записи из Контак...
506      Куда подевалось вход по отпечатку? При запуске...
17032    Все было хорошо недавно перестало работать из....
11215    Не устанавливается приложение!Раньше все было ...
3582     Установил приложение. Зарегистрироваться так и...
20173    У меня тоже телефон meizu pro 6. И тоже пишет ...
9355     Почему у сбера переводы длятся вечность? Польз...
8580     Здравствуйте. После последнего обновления стал...
1742     При регистрации не могу ввести разовый пароль ...
456      Разрабы тупые рукожопы! Наковыряли в обновлени...
15358    Не могу войти на свой счет и зарегать новый не...
20115    Очень медленное приложение в начале загрузки, .

## 1. Обработка текстов

In [8]:
!pip install pymorphy2

In [9]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

from pymorphy2 import MorphAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ограничимся минимальной очисткой текстов - только от точек, запятых и предлогов:

In [10]:
punctuation = r'[,.]'
morpher = MorphAnalyzer()

stop = {'без', 'безо', 'близ', 'в', 'во', 'вместо', 'вне',
        'для', 'до', 'за', 'из', 'изо', 'из-за', 'из-под', 
        'к', 'ко', 'кроме', 'между', 'меж', 'на', 'над', 'надо',
        'о', 'об', 'обо', 'от', 'ото', 'перед', 'передо', 'пред', 'предо',
        'пo', 'под', 'подо', 'при', 'про', 'ради', 'с', 'со',
        'сквозь', 'среди', 'у', 'через', 'чрез'}

Функции для обработки текстов:

In [11]:
def normalize_text(text_list):
    return [morpher.parse(word)[0].normal_form \
            for word in text_list if word not in stop]

def preprocess_text(text):    
    text_list = nltk.word_tokenize(str(text))
    text_list = normalize_text(text_list)
    return text_list

Обработка текстов:

In [53]:
data['Text'] = data['Content'].str.lower()
data['Text'] = data['Text'].str.replace(r'[.,]', ' ')
data['Text'] = [preprocess_text(text_list) for text_list in tqdm(data['Text'])]

100%|██████████| 19748/19748 [00:31<00:00, 633.55it/s]


Датафрейм с обработанными текстами:

In [13]:
data['Text'] = data['Text'].str.join(' ')
data.head(n=10)

,Rating,Content,Date,target,Text
0,5,It just works!,2017-08-14,1,it just works !
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,1,отлично весь
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,1,стать зависать 1 % работа антивирус далёкий ни...
4,5,"Очень удобно, работает быстро.",2017-08-14,1,очень удобно работать быстро
5,5,Всё удобно норм 👍👍👍,2017-08-14,1,весь удобно норма 👍👍👍
6,5,Очень удобное приложение.,2017-08-14,1,очень удобный приложение
7,5,Все устраивает,2017-08-14,1,весь устраивать
8,5,У меня работает все четко. В отличии от банком...,2017-08-14,1,я работать весь чётко отличие банкомат который...
9,5,Очень все хорошо👍,2017-08-14,1,очень весь хорошо👍


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = data['Text']
y = data['target']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

In [17]:
max_words = 5000
emb_len = 128
max_len = 40

In [18]:
train_corpus = ' '.join(X_train)
tokens = word_tokenize(train_corpus)

In [19]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [20]:
# from nltk.probability import FreqDist
dist = nltk.probability.FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [21]:
tokens_filtered_top[:10]

['приложение',
 'весь',
 'не',
 'и',
 'очень',
 'удобно',
 'я',
 'работать',
 'удобный',
 'что']

In [22]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [23]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [24]:
X_train_sec = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test_sec = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [25]:
X_train_sec.shape

(14811, 40)

In [26]:
X_train_sec[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 13,  8, 11], dtype=int32)

In [27]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten

In [43]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=emb_len, input_length=max_len))
model.add(Conv1D(emb_len, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [44]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [45]:
history1 = model.fit(X_train_sec, y_train,
                     batch_size=10,
                     epochs=10,
                     verbose=1,
                     validation_split=0.25)

Epoch 1/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.2315 - acc: 0.8934 - val_loss: 0.1744 - val_acc: 0.9271
Epoch 2/10
1111/1111 [==============================] - 13s 11ms/step - loss: 0.1562 - acc: 0.9269 - val_loss: 0.1941 - val_acc: 0.9263
Epoch 3/10
1111/1111 [==============================] - 13s 11ms/step - loss: 0.1309 - acc: 0.9481 - val_loss: 0.2005 - val_acc: 0.9255
Epoch 4/10
1111/1111 [==============================] - 13s 11ms/step - loss: 0.1067 - acc: 0.9591 - val_loss: 0.2362 - val_acc: 0.9176
Epoch 5/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.0839 - acc: 0.9674 - val_loss: 0.3467 - val_acc: 0.9125
Epoch 6/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.0693 - acc: 0.9732 - val_loss: 0.2875 - val_acc: 0.9184
Epoch 7/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.0606 - acc: 0.9779 - val_loss: 0.3544 - val_acc: 0.9174
Epoch 8/10
1111/1111 [==========================

In [46]:
loss, acc = model.evaluate(X_test_sec, y_test, batch_size=10, verbose=1)
print(loss, acc)

494/494 [==============================] - 1s 2ms/step - loss: 0.4600 - acc: 0.9165
0.46003401279449463 0.916548490524292


In [32]:
from gensim.models import Word2Vec

In [33]:
X_train_tokenized = X_train.apply(word_tokenize)

In [34]:
sentences = X_train_tokenized.tolist()
params = {'size': emb_len, 'window': 5, 'min_count': 2, 'sg': 1, 'hs': 0, 'negative': 10, 'workers': 32, 'seed': 34}

In [35]:
%%time
model_w2v = Word2Vec(**params)
model_w2v.build_vocab(sentences)
model_w2v.train(sentences,
                total_examples=len(sentences),
                epochs=20)

CPU times: user 20.3 s, sys: 153 ms, total: 20.5 s
Wall time: 11 s


In [37]:
embs = [np.zeros(emb_len).reshape((1, emb_len))]
for word in vocabulary:
    if word in model_w2v:
        emb = model_w2v[word].reshape((1, emb_len))
    else:
        emb = np.zeros(emb_len).reshape((1, emb_len))
    embs.append(emb)
embs_ = np.concatenate(embs).astype(np.float32)

In [38]:
embeddings_initializer = keras.initializers.Constant(embs_)

In [50]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=emb_len, input_length=max_len, embeddings_initializer=embeddings_initializer))
model.add(Conv1D(emb_len, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [51]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [52]:
history2 = model.fit(X_train_sec, y_train,
                     batch_size=10,
                     epochs=10,
                     verbose=1,
                     validation_split=0.25)

Epoch 1/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.2245 - acc: 0.9090 - val_loss: 0.1655 - val_acc: 0.9241
Epoch 2/10
1111/1111 [==============================] - 13s 11ms/step - loss: 0.1683 - acc: 0.9346 - val_loss: 0.1605 - val_acc: 0.9338
Epoch 3/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.1371 - acc: 0.9478 - val_loss: 0.1614 - val_acc: 0.9301
Epoch 4/10
1111/1111 [==============================] - 14s 12ms/step - loss: 0.1108 - acc: 0.9633 - val_loss: 0.2049 - val_acc: 0.9328
Epoch 5/10
1111/1111 [==============================] - 13s 11ms/step - loss: 0.0871 - acc: 0.9721 - val_loss: 0.2465 - val_acc: 0.9220
Epoch 6/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.0649 - acc: 0.9818 - val_loss: 0.3326 - val_acc: 0.9230
Epoch 7/10
1111/1111 [==============================] - 13s 12ms/step - loss: 0.0501 - acc: 0.9865 - val_loss: 0.3312 - val_acc: 0.9163
Epoch 8/10
1111/1111 [==========================

In [42]:
loss, acc = model.evaluate(X_test_sec, y_test, batch_size=10, verbose=1)
print(loss, acc)

494/494 [==============================] - 1s 2ms/step - loss: 0.1954 - acc: 0.9261
0.19538545608520508 0.9260684847831726
